<a href="https://colab.research.google.com/github/rameshbabulakshmanan84/AI-Day---Knowledge-Graph/blob/master/NER_ADE_Bluebert_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
def readfile():
  path='/content/drive/MyDrive/NLP/ADE/Dataset/ADEtrain.tsv'
  f=open(path)
  data1=[]
  data = []
  sentence=[]
  label=[]
  for line in f:
  #print(line)
  #print(len(line))
    line1=line.split('\t')
  #print(len(line1))
  #print(line1)
    if line1[0]=="\n" or len(line1)==0:
      if len(sentence) >0:
      #print("Sentence Length",)
    #if len(sentence) != 0:
      #print("New line")
    #print(line1)
        data1.append((sentence,label))
        data=[]
        sentence=[]
        label=[]
      continue
    #else:
      #continue
    elif len(line1)<=5:    
      #print("other")
      #print(line1)
      continue 
    else:
    #line1=line.split('\t')
      #print(line1)
      sentence.append(line1[5])
      label.append(line1[1])
  return data1    

In [3]:
#Call the functiona
data1=readfile()

In [4]:
#Print a sample data
data1[0]

(['Intravenous', 'azithromycin', '-', 'induced', 'ototoxicity', '.'],
 ['O', 'S-DRUG', 'O', 'O', 'S-AE', 'O'])

In [5]:
#Length of the sentence
leng=[]
for i in range(len(data1)):
  leng.append(len(data1[i][0]))
#print(len(leng))
print(sum(leng)/len(leng))


21.599762752075918


In [6]:
import os
os.environ['CUDA_LAUNCH_BLOCKING']='1'

In [7]:
import pandas as pd
import csv

In [8]:
tags=['O','B-AE','I-AE','E-AE','S-AE','B-DRUG','I-DRUG','E-DRUG','S-DRUG','PAD']
labels_map={}
for i,t in enumerate(tags):
  labels_map[t]=i
print(labels_map)

{'O': 0, 'B-AE': 1, 'I-AE': 2, 'E-AE': 3, 'S-AE': 4, 'B-DRUG': 5, 'I-DRUG': 6, 'E-DRUG': 7, 'S-DRUG': 8, 'PAD': 9}


In [9]:
import pandas as pd
import numpy as np
from tqdm import tqdm, trange

In [10]:
!pip3 install transformers

     |████████████████████████████████| 1.5MB 6.0MB/s 
     |████████████████████████████████| 890kB 51.9MB/s 
     |████████████████████████████████| 2.9MB 45.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=2749a10c151535e77b1161ed15c853dadcfc020157548931022151b744ef527a
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [11]:
#Apply the Bert
#use pytorch implementation of bert
import torch
from torch.utils.data import TensorDataset,DataLoader,RandomSampler,SequentialSampler
from transformers import AutoTokenizer,AutoModelForTokenClassification,AutoModel
import logging
logging.basicConfig(format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)
logger = logging.getLogger(__name__)

In [12]:
#download the biobert models from dmis lab - korea univ
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("bionlp/bluebert_pubmed_uncased_L-12_H-768_A-12")
model = AutoModel.from_pretrained("bionlp/bluebert_pubmed_uncased_L-12_H-768_A-12")

12/20/2020 15:11:31 - INFO - filelock -   Lock 140400100113208 acquired on /root/.cache/huggingface/transformers/7771a8dffb22ec7e4587efbb0a2226c2219fb28e16396082cbb12fe45f3c3287.67d46473a54abe2f4364f5944343ffc7f5797c5712cf84dfa410256786ac0052.lock


12/20/2020 15:11:31 - INFO - filelock -   Lock 140400100113208 released on /root/.cache/huggingface/transformers/7771a8dffb22ec7e4587efbb0a2226c2219fb28e16396082cbb12fe45f3c3287.67d46473a54abe2f4364f5944343ffc7f5797c5712cf84dfa410256786ac0052.lock
12/20/2020 15:11:31 - INFO - filelock -   Lock 140401413533880 acquired on /root/.cache/huggingface/transformers/f70a67bfc2fef6768bcf4df654d68466d55736dc7582684b08b673dd05634364.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99.lock


12/20/2020 15:11:31 - INFO - filelock -   Lock 140401413533880 released on /root/.cache/huggingface/transformers/f70a67bfc2fef6768bcf4df654d68466d55736dc7582684b08b673dd05634364.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99.lock


12/20/2020 15:11:32 - INFO - filelock -   Lock 140398846061928 acquired on /root/.cache/huggingface/transformers/32d1b181bbb83072d7b612edb8ed5a530cbb4ec01f31d0cec34ad1e86eecc25f.0dce6ec9cc2af6994450c367bae9b09cd312262205981e711ba8cae4f9a9332a.lock


12/20/2020 15:11:43 - INFO - filelock -   Lock 140398846061928 released on /root/.cache/huggingface/transformers/32d1b181bbb83072d7b612edb8ed5a530cbb4ec01f31d0cec34ad1e86eecc25f.0dce6ec9cc2af6994450c367bae9b09cd312262205981e711ba8cae4f9a9332a.lock


In [13]:
#convert the sequences to tokens 
sentences=[]
label=[]
for i in data1:
  sentence=i[0]
  tags=i[1]
  tokenized_sentence=[]
  labels=[]
  for word,tag in zip(sentence,tags):      
    tokenized_word=tokenizer.tokenize(word)
    #print(tokenized_word)
    n_subwords=len(tokenized_word)
    tokenized_sentence.extend(tokenized_word)
    labels.extend([tag] * n_subwords)
  sentences.append(tokenized_sentence)
  label.append(labels)

In [14]:
print(len(sentences))

4215


In [15]:
print(len(label))

4215


In [16]:
#print a sample sentence
sentences[5]

['results',
 ':',
 'a',
 '44',
 '-',
 'year',
 '-',
 'old',
 'man',
 'taking',
 'nap',
 '##ro',
 '##x',
 '##en',
 'for',
 'chronic',
 'low',
 'back',
 'pain',
 'and',
 'a',
 '20',
 '-',
 'year',
 '-',
 'old',
 'woman',
 'on',
 'ox',
 '##ap',
 '##ro',
 '##zin',
 'for',
 'r',
 '##he',
 '##uma',
 '##to',
 '##id',
 'arthritis',
 'presented',
 'with',
 'tense',
 'bull',
 '##ae',
 'and',
 'cut',
 '##aneous',
 'fra',
 '##gil',
 '##ity',
 'on',
 'the',
 'face',
 'and',
 'the',
 'back',
 'of',
 'the',
 'hands',
 '.']

In [17]:
#print a sample label
label[5]

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'S-DRUG',
 'S-DRUG',
 'S-DRUG',
 'S-DRUG',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'S-DRUG',
 'S-DRUG',
 'S-DRUG',
 'S-DRUG',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-AE',
 'E-AE',
 'E-AE',
 'O',
 'B-AE',
 'B-AE',
 'E-AE',
 'E-AE',
 'E-AE',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']

In [18]:
#pad_sequences is used to ensure that all sequences in a list have the same length
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix

In [19]:
#setting the maximum size to be 128 since only few sequences have above 128
MAX_LEN = 128
#keep the batch size as 16 
Batch_Size = 16

In [20]:
#Convert tokens to ids and pad with 0's till the max seq length of 128
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in sentences],maxlen=MAX_LEN, dtype="long", value=0.0,
                          truncating="post", padding="post")

In [21]:
label[2][5]

'S-AE'

In [22]:
label[2][5]='B-AE'

In [ ]:
p=[[labels_map.get(l) for l in lab] for lab in label]
p[2]

[0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [23]:
#Convert tags to ids and pad with 0's till the max seq length of 128
tags = pad_sequences([[labels_map.get(l) for l in lab] for lab in label],
                     maxlen=MAX_LEN, value=labels_map["PAD"], padding="post",
                     dtype="long", truncating="post")

In [24]:
tags[56]

array([0, 0, 0, 0, 0, 8, 0, 0, 0, 0, 0, 1, 3, 3, 3, 0, 4, 4, 4, 4, 0, 0,
       0, 4, 4, 4, 0, 0, 0, 8, 0, 0, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9,
       9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9,
       9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9,
       9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9,
       9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9])

In [25]:
input_ids[56]

array([ 4725,  1024,  1996,  5776,  3223, 22597,  4078,  6132, 25090,
        9276,  2005,  5729, 24471, 22723,  4360,  1010, 17076,  3695,
       14728,  2863,  1010,  1998,  8138,  1059, 21030,  6774,  4525,
        2013,  2014, 22597, 13004,  1012,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0]

In [26]:
#attenton masks to ensure padding trainin is not impacted
attention_masks = [[float(i!= 0.0) for i in ii] for ii in input_ids]

In [27]:
tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(input_ids,tags,random_state=2018, test_size=0.3)
tr_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids,random_state=2018, test_size=0.3)

In [28]:
#convert all tensor format
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)

In [29]:
val_inputs=val_inputs[:1264]
val_tags=val_tags[:1264]
val_masks=val_masks[:1264]

In [30]:
val_inputs.shape

torch.Size([1264, 128])

In [31]:
#train_data in the form - input tokens, attention masks for the corresponding tokens and the tags for the tokens
train_data = TensorDataset(tr_inputs,tr_masks,tr_tags)
#returns random indices from the data source
train_sampler = RandomSampler(train_data)
#dataloader combines the dataset and the sampler 
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=Batch_Size)
valid_data = TensorDataset(val_inputs, val_masks, val_tags)
#samples elements sequentially in the same order
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data,sampler=valid_sampler, batch_size=Batch_Size)

In [32]:
val_inputs.shape

torch.Size([1264, 128])

In [33]:
import transformers
#bertfortokenclassification addsa classifier on top of bert model
from transformers import AutoModelForTokenClassification,AdamW,AutoConfig

In [34]:
print(len(labels_map))

10


In [35]:
from transformers import BertForTokenClassification,AdamW,AutoConfig

model2 = BertForTokenClassification.from_pretrained("bionlp/bluebert_pubmed_uncased_L-12_H-768_A-12",num_labels=len(labels_map), output_attentions = False,output_hidden_states = False)

Some weights of the model checkpoint at bionlp/bluebert_pubmed_uncased_L-12_H-768_A-12 were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification 

In [36]:
import torch
#specify GPU as device
if torch.cuda.is_available():
  device=torch.device("cuda")
  print('There are %d GPU(s) available.' % torch.cuda.device_count())
  print('We will use the GPU:', torch.cuda.get_device_name(0))

There are 1 GPU(s) available.
We will use the GPU: Tesla P4


In [ ]:
# Tell pytorch to run this model on the GPU.
model2.cuda()

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [37]:
model2.to(device)

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [38]:
#grab the optimzer
optimizer=AdamW(model2.parameters(),
               lr=2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5 
               eps=1e-8) #a very small number to prevent any division by zero in the implementation)

In [39]:
from transformers import get_linear_schedule_with_warmup

In [66]:
epochs = 7

In [67]:
total_steps = len(train_dataloader) * epochs

In [68]:
#Create a schedule with a learning rate that decreases linearly from the initial lr set in the optimizer to 0, after a warmup period during which it increases linearly from 0 to the initial lr set in the optimizer.

scheduler=get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,#deafault value
    num_training_steps=total_steps)


In [69]:
from sklearn.metrics import f1_score, accuracy_score,confusion_matrix

In [70]:
import random
import numpy as np

In [71]:
# Set the seed value all over the place to make this reproducible.
seed_val=42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [72]:
def get_special_tokens(tokenizer, labels_map):
  pad_tok = tokenizer.vocab["[PAD]"]
  sep_tok = tokenizer.vocab["[SEP]"]
  cls_tok = tokenizer.vocab["[CLS]"]
  o_lab = labels_map["O"]
  return pad_tok, sep_tok, cls_tok, o_lab

In [73]:
def flat_accuracy(valid_tags, pred_tags):

    """
    Define a flat accuracy metric to use while training the model.
    """

    return (np.array(valid_tags) == np.array(pred_tags)).mean()

In [74]:
def annot_confusion_matrix(valid_tags, pred_tags):

    """
    Create an annotated confusion matrix by adding label
    annotations and formatting to sklearn's `confusion_matrix`.
    """

    # Create header from unique tags
    header = sorted(list(set(valid_tags + pred_tags)))

    # Calculate the actual confusion matrix
    matrix = confusion_matrix(valid_tags, pred_tags, labels=header)

    # Final formatting touches for the string output
    mat_formatted = [header[i] + "\t" + str(row) for i, row in enumerate(matrix)]
    content = "\t" + " ".join(header) + "\n" + "\n".join(mat_formatted)

    return content


In [75]:
taglist=['O','B-AE','I-AE','E-AE','S-AE','B-DRUG','I-DRUG','E-DRUG','S-DRUG','PAD']

In [76]:
#check --
import time
loss_values, validation_loss_values = [], []
training_stats = []
tic= time.time()

pad_tok, sep_tok, cls_tok, o_lab = get_special_tokens(tokenizer, labels_map)

for _ in trange(epochs,desc="EPOCH"):
  print("")
  #print ---Epoch 1/2 ----
  #print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
  print('Training...')

 
  #t0 = time.time()
  #train the model
  model2.train()
  tr_loss, tr_accuracy = 0,0
  nb_tr_examples, nb_tr_steps = 0,0
  tr_preds, tr_labels = [], []

  for nb_tr_steps,batch in enumerate(train_dataloader):
    print(nb_tr_steps)
    #print(batch)

    # Unpack this training batch from our dataloader and load to GPU
    b_input_ids = batch[0].to(device) # input ids
    b_input_mask = batch[1].to(device) # attention masks to performing attention on padding token indices
    b_labels = batch[2].to(device) # labels

    #always clear the gradients before doing a backward pass
    model2.zero_grad()
   
    #print("inputids",b_input_ids)
    #print("labels",b_labels)
    #print("Mask",b_input_mask)



    #perform a forward pass
    output = model2(b_input_ids,token_type_ids=None,attention_mask=b_input_mask,labels=b_labels)

    #print(output)

    #print("loss",logits1)
    #print("loss",output[0])
    #print("logit",output[1])
    #print("logits",loss)
    loss=output[0]
    logits=output[1]

    #Perform backward loss
    loss.backward()

    #print(loss.item())
    #Add the loss
    tr_loss += loss.item()
    #Compute the totale number of examples based on the batch size
    nb_tr_examples += b_input_ids.size(0)
    nb_tr_steps += 1

     # Subset out unwanted predictions on CLS/PAD/SEP tokens
    preds_mask = ((b_input_ids != cls_tok) & (b_input_ids != pad_tok) & (b_input_ids != sep_tok))

    pred_mask=preds_mask.to("cpu").numpy()

     #logits = logits.detach().cpu().numpy()
    tr_logits = logits.detach().cpu().numpy()

    tr_label_ids = torch.masked_select(b_labels, (preds_mask == 1))
    tr_batch_preds = np.argmax(tr_logits[pred_mask.squeeze()], axis=1)
    tr_batch_labels = tr_label_ids.to("cpu").numpy()

    tr_preds.extend(tr_batch_preds)
    tr_labels.extend(tr_batch_labels)

    # Compute training accuracy
    tmp_tr_accuracy = flat_accuracy(tr_batch_labels, tr_batch_preds)


    tr_accuracy += tmp_tr_accuracy

    torch.nn.utils.clip_grad_norm_(model2.parameters(), 1.0)


    #Update the weights
    optimizer.step()

    #Update the learning rate
    scheduler.step()
  
  #Compute the avergae loss
  tr_loss = tr_loss / nb_tr_steps
  #compute the average accuracy
  tr_accuracy = tr_accuracy / nb_tr_steps

  #Print the computed metrics for training after the epochs
  print(f"Train loss: {tr_loss}")
  print(f"Train accuracy: {tr_accuracy}")
  print(" Training is completed")
  print("Running Validation... ")


  ##############################
    ##### VALIDATION STARTS #####
    ############################  

  model2.eval() # getting the trained model to evaluation mode
  # Tracking variables 
  eval_loss, eval_accuracy = 0, 0
  nb_eval_steps, nb_eval_examples = 0, 0
  predictions, true_labels = [], []

 
  for batch in valid_dataloader:
    b_input_ids = batch[0].to(device) # input ids
    b_input_mask = batch[1].to(device) # attention masks
    b_labels = batch[2].to(device) # labels

    with torch.no_grad():
      output=model2(b_input_ids,token_type_ids=None,attention_mask=b_input_mask,labels=b_labels)

      tmp_eval_loss,logits = output[:2]

    # Subset out unwanted predictions on CLS/PAD/SEP tokens
    preds_mask = ((b_input_ids != cls_tok) & (b_input_ids != pad_tok) & (b_input_ids != sep_tok))

    #Derive the ligits to cpu
    logits = logits.detach().cpu().numpy()


    label_ids = torch.masked_select(b_labels,(preds_mask == 1))

    #Move the mask to cpu 
    preds_mask=preds_mask.to("cpu").numpy()



    #np.argmax 
    val_batch_preds =np.argmax(logits[preds_mask.squeeze()],axis=1)


    val_batch_labels = label_ids.to("cpu").numpy()

    predictions.extend(val_batch_preds)
    true_labels.extend(val_batch_labels)

    #compute the accuracy
    tmp_eval_accuracy = flat_accuracy(val_batch_labels, val_batch_preds)
    
    eval_loss += tmp_eval_loss.mean().item()
    eval_accuracy += tmp_eval_accuracy
     
    nb_eval_examples += b_input_ids.size(0)
    nb_eval_steps += 1  

  pred_tags = [taglist[i] for i in predictions]
  valid_tags = [taglist[i] for i in true_labels]
  cl_report = classification_report(valid_tags, pred_tags)
  conf_mat = annot_confusion_matrix(valid_tags, pred_tags)
  eval_loss = eval_loss / nb_eval_steps
  eval_accuracy = eval_accuracy / nb_eval_steps

  # Report metrics
  print(f"Validation loss: {eval_loss}")
  print(f"Validation Accuracy: {eval_accuracy}")
  print(f"Classification Report:\n {cl_report}")
  print(f"Confusion Matrix:\n {conf_mat}")




   

EPOCH:   0%|          | 0/7 [00:00<?, ?it/s]


Training...
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
Train loss: 0.027276346470053132
Train accuracy: 0.9935360524709967
 Training is completed
Running Validation... 


EPOCH:  14%|█▍        | 1/7 [01:48<10:48, 108.03s/it]

Validation loss: 0.3401366122349908
Validation Accuracy: 0.9335502062249451
Classification Report:
               precision    recall  f1-score   support

        B-AE       0.76      0.89      0.82      2189
      B-DRUG       0.88      0.93      0.90       527
        E-AE       0.79      0.92      0.85      2228
      E-DRUG       0.91      0.93      0.92       501
        I-AE       0.77      0.79      0.78      1688
      I-DRUG       0.93      0.85      0.89       247
           O       0.98      0.94      0.96     25763
        S-AE       0.91      0.86      0.88      2114
      S-DRUG       0.93      0.98      0.96      4748

    accuracy                           0.93     40005
   macro avg       0.87      0.90      0.89     40005
weighted avg       0.94      0.93      0.93     40005

Confusion Matrix:
 	B-AE B-DRUG E-AE E-DRUG I-AE I-DRUG O S-AE S-DRUG
B-AE	[1952    0    0    0   89    0  116   11   21]
B-DRUG	[  4 488   0   0   1   3  12   0  19]
E-AE	[   4    0 2043    1   

EPOCH:  29%|██▊       | 2/7 [03:37<09:02, 108.51s/it]

...

Validation loss: 0.37308687895913667
Validation Accuracy: 0.9371153130059217
Classification Report:
               precision    recall  f1-score   support

        B-AE       0.81      0.86      0.83      2189
      B-DRUG       0.88      0.95      0.91       527
        E-AE       0.83      0.90      0.87      2228
      E-DRUG       0.92      0.95      0.93       501
        I-AE       0.77      0.81      0.79      1688
      I-DRUG       0.91      0.89      0.90       247
           O       0.97      0.95      0.96     25763
        S-AE       0.90      0.87      0.89      2114
      S-DRUG       0.94      0.98      0.96      4748

    accuracy                           0.94     40005
   macro avg       0.88      0.91      0.89     40005
weighted avg       0.94      0.94      0.94     40005

Confusion Matrix:
 	B-AE B-DRUG E-AE E-DRUG I-AE I-DRUG O S-AE S-DRUG
B-AE	[1887    0    0    0  120    0  147   12   23]
B-DRUG	[  4 500   0   0   0   4  10   0   9]
E-AE	[   1    0 2008    1  

EPOCH:  86%|████████▌ | 6/7 [10:55<01:49, 109.37s/it]

Validation loss: 0.36356148095447804
Validation Accuracy: 0.9394885086372592
Classification Report:
               precision    recall  f1-score   support

        B-AE       0.81      0.87      0.84      2189
      B-DRUG       0.87      0.94      0.90       527
        E-AE       0.83      0.91      0.87      2228
      E-DRUG       0.91      0.95      0.93       501
        I-AE       0.78      0.81      0.80      1688
      I-DRUG       0.94      0.85      0.89       247
           O       0.98      0.95      0.97     25763
        S-AE       0.91      0.87      0.89      2114
      S-DRUG       0.94      0.98      0.96      4748

    accuracy                           0.94     40005
   macro avg       0.88      0.90      0.89     40005
weighted avg       0.94      0.94      0.94     40005

Confusion Matrix:
 	B-AE B-DRUG E-AE E-DRUG I-AE I-DRUG O S-AE S-DRUG
B-AE	[1909    4    0    0  117    0  135   12   12]
B-DRUG	[  5 494   0   0   0   2  11   0  15]
E-AE	[   1    0 2029    1  

EPOCH: 100%|██████████| 7/7 [12:45<00:00, 109.35s/it]

Validation loss: 0.36911798334574397
Validation Accuracy: 0.9389247843851547
Classification Report:
               precision    recall  f1-score   support

        B-AE       0.81      0.88      0.84      2189
      B-DRUG       0.86      0.94      0.90       527
        E-AE       0.83      0.91      0.87      2228
      E-DRUG       0.91      0.95      0.93       501
        I-AE       0.77      0.82      0.80      1688
      I-DRUG       0.94      0.84      0.89       247
           O       0.98      0.95      0.96     25763
        S-AE       0.91      0.86      0.89      2114
      S-DRUG       0.94      0.98      0.96      4748

    accuracy                           0.94     40005
   macro avg       0.88      0.90      0.89     40005
weighted avg       0.94      0.94      0.94     40005

Confusion Matrix:
 	B-AE B-DRUG E-AE E-DRUG I-AE I-DRUG O S-AE S-DRUG
B-AE	[1917    4    0    0  120    0  124   13   11]
B-DRUG	[  5 494   0   0   0   2  11   0  15]
E-AE	[   1    0 2037    1  

In [77]:
test_sentence="A 44-year-old man taking naproxen for chronic low back pain and a 20-year-old woman on oxaprozin for rheumatoid arthritis presented with tense bullae and cutaneous fragility on the face and the back of the hands."






In [78]:
#Check for tokenizer - biobert
def evaluate1(test_sentence):
  tokenized_sentence=tokenizer.encode(test_sentence)
  #conver to tensor format
  input_ids = torch.tensor([tokenized_sentence])
  input_ids=input_ids.cuda()
  print(input_ids)
  #run the sentence through the model
  with torch.no_grad():
    output=model2(input_ids)
  label_indices=torch.argmax(output[0],axis=2)
  tokens=tokenizer.convert_ids_to_tokens(input_ids[0])
  tags=['O','B-AE','I-AE','E-AE','S-AE','B-DRUG','I-DRUG','E-DRUG','S-DRUG','PAD']
  new_token,new_labels=[],[]
  for token,label_idx in zip(tokens,label_indices[0]):
  #print(token)
  #print(tags[label_idx])
    if token.startswith("##"):
    #add at the end
      new_token[-1]=new_token[-1]+token[2:]
    else:
    #if its not special token
      new_labels.append(tags[label_idx])
      new_token.append(token)
  for token,label in zip(new_token,new_labels):
    print("{}\t{}".format(token,label))   

In [79]:
evaluate1(test_sentence)

tensor([[  101,  1037,  4008,  1011,  2095,  1011,  2214,  2158,  2635, 18996,
          3217,  2595,  2368,  2005, 11888,  2659,  2067,  3255,  1998,  1037,
          2322,  1011,  2095,  1011,  2214,  2450,  2006, 23060,  9331,  3217,
         17168,  2005,  1054,  5369, 12248,  3406,  3593, 27641,  3591,  2007,
          9049,  7087,  6679,  1998,  3013, 17191, 25312, 20142,  3012,  2006,
          1996,  2227,  1998,  1996,  2067,  1997,  1996,  2398,  1012,   102]],
       device='cuda:0')
[CLS]	O
a	O
44	O
-	O
year	O
-	O
old	O
man	O
taking	O
naproxen	S-DRUG
for	O
chronic	O
low	O
back	O
pain	O
and	O
a	O
20	O
-	O
year	O
-	O
old	O
woman	O
on	O
oxaprozin	S-DRUG
for	O
rheumatoid	O
arthritis	O
presented	O
with	O
tense	B-AE
bullae	S-AE
and	O
cutaneous	B-AE
fragility	E-AE
on	E-AE
the	O
face	E-AE
and	O
the	O
back	O
of	O
the	O
hands	E-AE
.	O
[SEP]	O


In [80]:
model2.save_pretrained('/content/drive/MyDrive/NLP/ADE/Model - ADE - BlueBert_v1')

In [81]:
tokenizer.save_pretrained('/content/drive/MyDrive/NLP/ADE/Model - ADE - BlueBert_v1')

('/content/drive/MyDrive/NLP/ADE/Model - ADE - BlueBert_v1/tokenizer_config.json',
 '/content/drive/MyDrive/NLP/ADE/Model - ADE - BlueBert_v1/special_tokens_map.json',
 '/content/drive/MyDrive/NLP/ADE/Model - ADE - BlueBert_v1/vocab.txt',
 '/content/drive/MyDrive/NLP/ADE/Model - ADE - BlueBert_v1/added_tokens.json')

In [82]:
tags=['O','B-AE','I-AE','E-AE','S-AE','B-DRUG','I-DRUG','E-DRUG','S-DRUG','PAD']

In [83]:
label_map = {i : label for i, label in enumerate(tags,1)}
print(label_map)

{1: 'O', 2: 'B-AE', 3: 'I-AE', 4: 'E-AE', 5: 'S-AE', 6: 'B-DRUG', 7: 'I-DRUG', 8: 'E-DRUG', 9: 'S-DRUG', 10: 'PAD'}


In [84]:
print(len(label_map))

10


In [85]:
bert_model='bionlp/bluebert_pubmed_uncased_L-12_H-768_A-12'

In [86]:
import json

In [87]:
label_map = {i : label for i, label in enumerate(tags,1)}
model_config={"bert_model":bert_model,"max_seq_length":128,"num_labels":len(label_map),"label_map":label_map}

In [88]:
model_config

{'bert_model': 'bionlp/bluebert_pubmed_uncased_L-12_H-768_A-12',
 'label_map': {1: 'O',
  2: 'B-AE',
  3: 'I-AE',
  4: 'E-AE',
  5: 'S-AE',
  6: 'B-DRUG',
  7: 'I-DRUG',
  8: 'E-DRUG',
  9: 'S-DRUG',
  10: 'PAD'},
 'max_seq_length': 128,
 'num_labels': 10}

In [89]:
json.dump(model_config,open(os.path.join('/content/drive/MyDrive/NLP/ADE/Model - ADE - BlueBert_v1',"model_config.json"),"w"))